# 6.5.2 Create Heamps for Weather Parameters

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

Note / Dependencies

Let's review how we will use our dependencies. We'll use Pandas to read our CSV file and create the locations and measurements from the DataFrame. We'll use gmaps and the API key to create heatmaps and the locations map, and we'll use the requests dependency to make a request to the Google Places JSON file. This will allow us to get hotel locations from the latitude and longitude of the city.

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Avarua,CK,2020-11-17 08:18:14,-21.21,-159.78,77.00,88,20,11.41
1,1,Chokurdakh,RU,2020-11-17 08:19:30,70.63,147.92,-7.85,91,100,4.94
2,2,Torbay,CA,2020-11-17 08:19:30,47.67,-52.73,48.00,93,90,23.04
3,3,Albany,US,2020-11-17 08:16:00,42.60,-73.97,37.00,63,65,4.63
4,4,Dingle,PH,2020-11-17 08:19:31,11.00,122.67,83.44,81,100,9.33


One caveat to using gmaps: The data we use for any mapping must be either an integer or a floating-point decimal number. Let's check the data types for the columns of our DataFrame.

In [3]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

# Create a Maximum Temperature Heatmap

First, tell gmaps to use your API key. You only need to configure gmaps to use your API key once.

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

#Google heatmaps do not plot negative numbers. If you have a maximum temperature that is less than 0 °F, 
#then you will get an InvalidWeightException error for this line of code: heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

Figure(layout=FigureLayout(height='420px'))

# Adjust Heatmap Zoom, Intensity, and Point Radius ^

First, add the geographic center of Earth in the form of latitude and longitude (30.0° N and 31.0° E). Also, add a zoom level so that only one map of Earth is shown.

If you review the gmaps documentation, you may notice there is a dissipation option for creating heatmaps that can be added to the gmaps.heat_layer() attribute.

The options for this are:

1. The default option for the dissipation is "True," so we need to set our "dissipation" to "False."
2. We can add max_intensity to make each measurement have a better gradient variance.
3. We can add point_radius to make each measurement radius larger.


In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

# 6.5.3 Get Vacation Criteria - Making the Map interactive

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 80


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Avarua,CK,2020-11-17 08:18:14,-21.21,-159.78,77.00,88,20,11.41
2,2,Torbay,CA,2020-11-17 08:19:30,47.67,-52.73,48.00,93,90,23.04
3,3,Albany,US,2020-11-17 08:16:00,42.60,-73.97,37.00,63,65,4.63
6,6,Ushuaia,AR,2020-11-17 08:16:09,-54.80,-68.30,41.00,80,75,27.51
7,7,Pitimbu,BR,2020-11-17 08:19:32,-7.47,-34.81,73.40,100,20,8.16
8,8,Bredasdorp,ZA,2020-11-17 08:19:32,-34.53,20.04,64.40,88,75,11.41
10,10,Jamestown,US,2020-11-17 08:16:32,42.10,-79.24,34.00,95,90,7.47
12,12,Ancud,CL,2020-11-17 08:19:33,-41.87,-73.82,48.99,93,0,6.93
13,13,Atuona,PF,2020-11-17 08:19:33,-9.80,-139.03,78.13,73,0,15.52
14,14,Hilo,US,2020-11-17 08:19:34,19.73,-155.09,73.40,88,90,6.93


In [12]:
preferred_cities_df.count()

City_ID       383
City          383
Country       379
Date          383
Lat           383
Lng           383
Max Temp      383
Humidity      383
Cloudiness    383
Wind Speed    383
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

# NEW SCHOOL: Adding a new column to a dataframe

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Avarua,CK,77.00,-21.21,-159.78,
2,Torbay,CA,48.00,47.67,-52.73,
3,Albany,US,37.00,42.60,-73.97,
6,Ushuaia,AR,41.00,-54.80,-68.30,
7,Pitimbu,BR,73.40,-7.47,-34.81,
8,Bredasdorp,ZA,64.40,-34.53,20.04,
10,Jamestown,US,34.00,42.10,-79.24,
12,Ancud,CL,48.99,-41.87,-73.82,
13,Atuona,PF,78.13,-9.80,-139.03,
14,Hilo,US,73.40,19.73,-155.09,


# 6.5.4

In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
      try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
      except (IndexError, IndentationError):
        print("Hotel not found... skipping.")

IndentationError: unexpected indent (<ipython-input-16-a6f81d65cfd4>, line 15)

In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


This is a new code block, so let's review its form and function.

First, the info_box_template variable is assigned to a multiline string using three quotes. The text inside the multiline string is HTML code. HTML code is defined by the opening and closing the angular brackets (e.g., <tag> and <tag/>). Angular brackets always come in pairs. The opening angular bracket is followed by some text inside, such as dl, dt, and dd. The closing angular bracket is preceded by a forward-slash ("/"). The text inside with the angular brackets is called a tag. We'll see more examples of HTML code in an upcoming module.

Here's what these tags mean:

The <dl> tag is a description list (dl).
The <dt> tag is a term or name in a description list that is nested under the <dl> tag.
The <dd> tag is used to define the term or name or <dt> tag.
If we were to write out these tags on paper, it would look like this.

Description List: <dl>
Description Term: <dt>
Description Definition: <dd>
Description Term: <dt>
Description Definition: <dd>


In [20]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))